In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
_stderr = sys.stderr
null = open(os.devnull,'wb')

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import loompy

# from scipy.sparse import csr_matrix, vstack
sc.settings.verbosity
sc.logging.print_header()

scanpy==1.9.3 anndata==0.9.1 umap==0.5.3 numpy==1.23.5 scipy==1.10.1 pandas==1.5.3 scikit-learn==1.2.2 statsmodels==0.13.5 pynndescent==0.5.10


In [ ]:
work_dir = '/content/drive/MyDrive/TFM/TFM/hysteresis_project/scenic_run/'

# scRNA-seq preprocessing using Scanpy

First, we preprocess the scRNA-seq side of the multiome datasets. Most importantly we will use this side of the data to annotate cell types.

Further on, to use the actual SCENIC+ analysis the raw count matrix will be used.

In [ ]:
path = work_dir + 'filtered_data'

adata = sc.read_mtx(
    path + 'matrix.mtx.gz').T  # transpose the matrix
adata.var_names = pd.read_csv(
    path + 'features.tsv.gz',
    header=None, sep='\t').iloc[:, 0]
adata.obs_names = pd.read_csv(
    path + 'barcodes.tsv.gz',
    header=None).iloc[:, 0]

In [ ]:
adata.var_names_make_unique()
adata.X

## Basic Quality Control

Keep cells with at least 200 genes expressed and only keep genes which are expressed in at least 3 cells. Also, cells with a maximum of 20000 genes expressed and minimum 750 counts

In [ ]:
sc.pp.filter_cells(adata, min_genes = 200)
sc.pp.filter_genes(adata, min_cells = 3)

In [ ]:
sc.pp.filter_cells(adata, max_counts = 20000)
sc.pp.filter_cells(adata, min_counts = 750)

In [ ]:
#Predicting doublets in the scRNA-seq

import scrublet as scr

scrub = scr.Scrublet(adata.X)
doublet_scores, predicted_doublets = scrub.scrub_doublets()


In [ ]:
##As the doublets estimation is 0.0%, it does not really filter anything, but if there were any, it would filter them. 
# Add predicted_doublets to adata.obs
adata.obs['predicted_doublets'] = predicted_doublets

# Filter adata based on predicted_doublets
adata = adata[adata.obs['predicted_doublets'] == False]

In [ ]:
#Compute different metrics (in this case, mitochondrial genes and ribosomal genes) with sc.pp_qc_metrics

#Mitochondrial genes for all the samples
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

#Ribosomal genes for all the samples
adata.var['Rp'] = adata.var_names.str.startswith('Rp')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['Rp'], percent_top=None, log1p=False, inplace=True)




In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_Rp'], jitter=0.4, multi_panel=True)

## Data Normalization

The data normalization and scalation is only for visualizing the data. For the SCENIC+ pipeline, we will use the raw count matrix. For that, first we save the non-normalized and non-scale AnnData object in the raw slot before continuing. 

In [ ]:
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)

## Cell type annotation

This is the difference between SCENIC and SCENIC+. With using the annotation in the scRNA-seq data, we can use it to annotate the cells and use then the scATAC-seq data to do a more accurate analysis.

In [ ]:
#Importing the tsv metadata file that we got from the Seurat object using the function Seurat_to_MM10X
import anndata as ad

metadata_df = pd.read_csv(work_dir + 'WT_metadata.tsv', sep="\t", index_col=0)

# Subset the reference DataFrame to only include the barcodes present in the query data
ref_df = metadata_df.loc[adata.obs_names.intersection(metadata_df.index)]

# Assign the reference cluster annotations to the 'cell_state' column of the adata object
adata.obs['cell_state'] = ref_df['cell_state']

In [ ]:
adata.obs

In [ ]:
adata.X

In [ ]:
#Subset the reference DataFrame to only include the barcodes present in the query data
ref_df = metadata_df.loc[adata.obs_names.intersection(metadata_df.index)]

#Assign the reference cluster annotations to the 'clusters' column of the adata object
adata.obs['S.Score'] = ref_df['S.Score']

#Assign the reference cluster annotations to the 'clusters' column of the adata object
adata.obs['G2M.Score'] = ref_df['G2M.Score']


In [ ]:
#Perform downstream analysis on the data to see the effect of cell cycle
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)

#Visualize the clustering
sc.pl.umap(adata, color=['G2M.Score', 'S.Score', 'cell_state'])

In [ ]:
#Regressing out the cell cycle
sc.pp.regress_out(adata, ['S.Score', 'G2M.Score'])

In [ ]:
#Perform downstream analysis on the corrected data
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)

#Visualize the clustering with the corrected data
sc.pl.umap(adata, color=['G2M.Score', 'S.Score', 'cell_state'])

In [ ]:
sc.tl.leiden(adata, resolution = 0.5, key_added = 'leiden_res_0.5')
sc.pl.umap(adata, color = 'leiden_res_0.5')

In [ ]:
#Cleaning the annotation by running a clustering and assigning clusters to cell subtypes based on the maximum overlap. 

tmp_df = adata.obs.groupby(['leiden_res_0.5', 'cell_state']).size().unstack(fill_value=0)
tmp_df = (tmp_df / tmp_df.sum(0)).fillna(0)
leiden_to_annotation = tmp_df.idxmax(1).to_dict()
leiden_to_annotation

In [ ]:
#As we can see, there are cell types that are in multiple clusters, so now we will group these clusters into one. Also, removing the spaces in this new annotation
leiden_to_annotation['4'] = 'H1 reduct'
leiden_to_annotation['1'] = 'H1 main' 
leiden_to_annotation = {cluster: leiden_to_annotation[cluster].replace(' ', '_') for cluster in leiden_to_annotation.keys()}
leiden_to_annotation

In [ ]:
#Adding the annotation to the AnnData object
adata.obs['cell_type'] = [leiden_to_annotation[cluster_id] for cluster_id in adata.obs['leiden_res_0.5']]
del(leiden_to_annotation)
del(tmp_df)

In [ ]:
adata.obs

In [ ]:
#Final UMAP
sc.pl.umap(adata, color = 'cell_type')

In [ ]:
adata.write(os.path.join(work_dir, 'scRNA/adata.h5ad'), compression='gzip')

Now the pre-processing of the scRNA-seq data has ben done. With this, we filtered the cells with bad counts and obtained only the high quality cells. Also we annotated the cell types and made a clustering analysis to see how the different cells group. Also, the cell annotation is used later in the SCENIC+ pipeline. 

SCENIC+ uses the raw gene expression counts (without normalization and scaling). For that, we used the adata.raw to still keep the raw data of our scRNA-seq. 

Now, the next part of the pre-processing will be to analyze the scATAC-seq data. For this, we will use the cell annotation done in the scRNA-seq data pre-processing. 

# scATAC-seq preprocessing using pycisTopic

Now, we preprocess the scATAC-seq side of the multiome dataset of EpRAS-wt cells. Now, we will generate pseudobulk ATAC-seq profiles per cell type and call peaks. These peaks then will be merged into a consensus peak-set, do qualityy control on the scATAC-seq barcodes and finally run topic modelling to find sets of co-accessible regions and impute chromatin accessibility. 

In [ ]:
import pandas as pd
import scipy.io as sio
import scipy.sparse as sp
import pycisTopic as ctp
import anndata as ad

In [ ]:
# Output directory
out_dir = work_dir + 'scATAC/'

In [ ]:
#Writing temp directory
tmp_dir = work_dir + 'tmp_dir/'

In [ ]:
import anndata as ad
# Load the cell annotation data from the .h5ad file
adata = ad.read(work_dir + 'scRNA/adata.h5ad')
cell_data = adata.obs
cell_data['sample_id'] = 'WT'
cell_data['cell_type'] = cell_data['cell_type'].astype(str) # set data type of the celltype column to str, otherwise the export_pseudobulk function will complain.
del(adata)

In [ ]:
# Path to fragments
fragments_dict = {'WT': out_dir + 'fragments/atac_fragments_good.tsv.gz'}

In [ ]:
# Get chromosome sizes (for hg38 here)
import pyranges as pr
import requests
target_url='http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes'
chromsizes=pd.read_csv(target_url, sep='\t', header=None)
chromsizes.columns=['Chromosome', 'End']
chromsizes['Start']=[0]*chromsizes.shape[0]
chromsizes=chromsizes.loc[:,['Chromosome', 'Start', 'End']]
# Exceptionally in this case, to agree with CellRangerARC annotations
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].replace('v', '.') for x in range(len(chromsizes['Chromosome']))]
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].split('_')[1] if len(chromsizes['Chromosome'][x].split('_')) > 1 else chromsizes['Chromosome'][x] for x in range(len(chromsizes['Chromosome']))]
chromsizes=pr.PyRanges(chromsizes)

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
bw_paths, bed_paths = export_pseudobulk(input_data = cell_data,
                 variable = 'cell_type',                                                                     # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
                 sample_id_col = 'sample_id',
                 chromsizes = chromsizes,
                 bed_path = os.path.join(out_dir, 'consensus_peak_calling/pseudobulk_bed_files/'),          # specify where pseudobulk_bed_files should be stored
                 bigwig_path = os.path.join(out_dir, 'consensus_peak_calling/pseudobulk_bw_files/'),        # specify where pseudobulk_bw_files should be stored
                 path_to_fragments = fragments_dict,                                                        # location of fragment fiels
                 n_cpu = 6,                                                                                 # specify the number of cores to use, we use ray for multi processing
                 normalize_bigwig = True,
                 remove_duplicates = True,
                 _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
                 split_pattern = '-')

In [ ]:
#Deactivate the virtual environment
!deactivate